# Agents

Agent is a modular and flexible component that can perform a variety of tasks by leveraging multiple tools and large language models (LLMs). Agents act as intermediaries that can dynamically select and execute actions based on the input they receive. They enable the creation of complex applications that require decision-making and multi-step processes.


In [2]:
%%capture
# update or install the necessary libraries
%pip install --upgrade openai --trusted.host pypi.org --trusted.host files.pythonhosted.org
%pip install --upgrade langchain --trusted.host pypi.org --trusted.host files.pythonhosted.org
%pip install --upgrade langchain_community --trusted.host pypi.org --trusted.host files.pythonhosted.org
%pip install --upgrade langchain_openai --trusted.host pypi.org --trusted.host files.pythonhosted.org
%pip install --upgrade langchainhub --trusted.host pypi.org --trusted.host files.pythonhosted.org
%pip install --upgrade tavily-python --trusted.host pypi.org --trusted.host files.pythonhosted.org
%pip install --upgrade python-dotenv --trusted.host pypi.org --trusted.host files.pythonhosted.org

TavilySearchResults is a valuable tool that connects your language model (LLM) to the web using the Tavily Search API.

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

## Prompt

**LangChain Hub**

Efficiently manage your LLM components with the LangChain Hub.

In [4]:
# Get the prompt to use - you can modify this!
# https://smith.langchain.com/hub/hwchase17/openai-functions-agent
# The hwchase17/openai-functions-agent is a popular prompt template from LangChain Hub 
# designed for creating AI agents that can use function calling capabilities with LLM models. 
# This prompt template provides a structured format that enables agents to understand when and
# how to call external tools or functions to complete complex tasks that require multiple 
# steps or external data access. 
prompt = hub.pull("hwchase17/openai-functions-agent")

C:\Users\vijay\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


## LLM

In [5]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("../.env")

# Set the OpenAI API key from environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,openai_api_key=openai_api_key)

## Tools

Tavily's Search API is a search engine built specifically for AI agents (LLMs), delivering real-time, accurate, and factual results at speed.

Generate Tavily API Key from this [link](https://app.tavily.com/home)

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv("../.env")

os.environ["TAVILY_API_KEY"] = os.getenv('TAVILY_API_KEY')

In [8]:
search = TavilySearchResults()
tools = [search]

C:\Users\vijay\AppData\Local\Temp\ipykernel_3688\102372591.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults()


## Agent

Agents are a powerful feature in LangChain that allow you to create intelligent systems capable of performing complex tasks by leveraging multiple tools and large language models (LLMs). Agents can dynamically select and execute actions based on the input they receive, enabling the creation of sophisticated applications that require decision-making and multi-step processes.

In [9]:
agent = create_openai_functions_agent(llm, tools, prompt)

In [10]:
result = agent.invoke({"input": "what's the weather in Bengaluru?", "intermediate_steps": []})

In [11]:
result.tool

'tavily_search_results_json'

In [12]:
result.tool_input

{'query': 'weather in Bengaluru'}

In [13]:
result

AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Bengaluru'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in Bengaluru'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Bengaluru"}', 'name': 'tavily_search_results_json'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 95, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BwiKPpxDM4TvKmGrnhj94afLGJVkk', 'service_tier': 'default', 'finish_reason': 'function_call', 'logprobs': None}, id='run--dde64f47-6f5b-4b75-9c36-b76dc1656cc5-0', usage_metadata={'input_tokens': 95, 'output_tokens': 21, 'total_tokens': 1

## Agent Executor

An agent executor is a component that manages the execution of agents, allowing them to perform tasks and interact with tools in a controlled manner.

In [14]:
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [15]:
agent_executor.invoke({"input": "what is the weather in Bengaluru?"})

{'input': 'what is the weather in Bengaluru?',
 'output': 'The weather in Bengaluru in July 2025 is expected to have temperatures ranging from 19°C to 26°C. There may be some rainfall during this time as well. If you need more detailed information, you can visit [this link](https://en.climate-data.org/asia/india/karnataka/bengaluru-4562/t/july-7/).'}

## Streaming

In [16]:
for step in agent_executor.stream({"input": "what is the weather in Bengaluru?"}):
    print(step)

{'actions': [AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Bengaluru'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in Bengaluru'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Bengaluru"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--d595ade2-001c-4b42-8011-ec47fcc00b72')])], 'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in Bengaluru"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call', 'model_name': 'gpt-3.5-turbo-0125', 'service_tier': 'default'}, id='run--d595ade2-001c-4b42-8011-ec47fcc00b72')]}
{'steps': [AgentStep(action=AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in Ben

# **Let's Do an Activity**

## **Objective**

Creating an Agent with Tavily Search and OpenAI for Real-Time Information Retrieval

## **Scenario**

How can you create a LangChain agent that utilizes Tavily's Search API to retrieve real-time information and generate responses using OpenAI's GPT-3.5-turbo model?

## **Steps**

* Initialize API Keys
* Import Required Libraries

  * Import the necessary libraries from LangChain, LangChain Community, and OpenAI.

* Configure the LLM

  * Set up the ChatOpenAI model with your OpenAI API key.

* Set Up Tavily Search

  * Initialize the Tavily Search tool with your Tavily API key.

* Create a Prompt Template
* Create the Agent
* Invoke the Agent